# 10-714 Homework 4

In this homework, you will leverage all of the components built in the last three homeworks to solve some modern problems with high performing network structures. We will start by adding a few new ops leveraging our new CPU/CUDA backends. Then, you will implement convolution, and a convolutional neural network to train a classifier on the CIFAR-10 image classification dataset. Then, you will implement recurrent and long-short term memory (LSTM) neural networks, and do word-level prediction language modeling on the Penn Treebank dataset.

As always, we will start by copying this notebook and getting the starting code.
Reminder: __you must save a copy in drive__.

In [1]:
# Code to set up the assignment
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/
# !mkdir -p 10714
# %cd /content/drive/MyDrive/10714
# !git clone https://github.com/dlsys10714/hw4.git
# %cd /content/drive/MyDrive/dls_hw4

# !pip3 install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git
# !pip3 install pybind11

Mounted at /content/drive
/content/drive/MyDrive/dls_hw4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/dlsys10714/mugrade.git to /tmp/pip-req-build-ccmac99_
  Running command git clone -q https://github.com/dlsys10714/mugrade.git /tmp/pip-req-build-ccmac99_
  Created wheel for mugrade: filename=mugrade-1.2-py3-none-any.whl size=3969 sha256=e77426a399d8fe78bc6f7457191c68450e8c2945e0c83538bf4e630a44e3907d
  Stored in directory: /tmp/pip-ephem-wheel-cache-y35nn6_7/wheels/3b/27/4a/35539c24fda8ad9b0903b5e7fd1bf16d682b75d0b402a299a5
Successfully built mugrade
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 23.3 MB/s 


In [1]:
%cd /home/erostrate9/PycharmProjects/needle

/home/erostrate9/PycharmProjects/needle


In [8]:
!make clean
!make

rm -rf build python/needle/backend_ndarray/ndarray_backend*.so
-- The C compiler identification is GNU 10.4.0
-- The CXX compiler identification is GNU 10.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python: /home/erostrate9/anaconda3/envs/dls/bin/python3.9 (found version "3.9.15") found components: Development Interpreter Development.Module Development.Embed 
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- Found pybind11: /home/erostrate9/anaconda3/envs/dls/include (found version "2.10.1")
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Pe

In [2]:
import sys
sys.path.append('./python')

In [32]:
# !rm -rf /content/drive/MyDrive/dls_hw4/data/cifar-10-batches-py

In [33]:
# # Download the datasets you will be using for this assignment
#
import urllib.request
import os
#
# !mkdir -p './data/ptb'
# # Download Penn Treebank dataset
# ptb_data = "https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb."
# for f in ['train.txt', 'test.txt', 'valid.txt']:
#     if not os.path.exists(os.path.join('./data/ptb', f)):
#         urllib.request.urlretrieve(ptb_data + f, os.path.join('./data/ptb', f))
#
# Download CIFAR-10 dataset
if not os.path.isdir("./data/cifar-10-batches-py"):
    urllib.request.urlretrieve("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", "./data/cifar-10-python.tar.gz")
    !tar -xvzf './data/cifar-10-python.tar.gz' -C './data'

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


To finish setting up the assignment, go ahead and fill in all the code in `python/needle/autograd.py` using your solution code from the previous homework.

## Part 1: ND Backend [10 pts]

Fill in the following classes in `python/needle/ops.py`:

- `PowerScalar`
- `EWiseDiv`
- `DivScalar`
- `Transpose`
- `Reshape`
- `BroadcastTo`
- `Summation`
- `MatMul`
- `Negate`
- `Log`
- `Exp`
- `ReLU`
- `LogSumExp`
- `Tanh` (new)
- `Stack` (new)
- `Split` (new)

Note that for most of these, you already wrote the solutions in the previous homework and you should not need to change your previous solution, however `TanhOp`, `Stack`, and `Split` are newly added. `Stack` concatenates same-sized tensors along a new axis, and `Split` undoes this operation. The gradients of the two operations can be written in terms of each other. We do not directly test `Split`, and only test the backward pass of `Stack` (for which we assume you used `Split`).

**Note:** You may want to make your Summation op support sums over multiple axes; you will likely need it for the backward pass of the BroadcastTo op if yours supports broadcasting over multiple axes at a time. However, this is more about ease of use than necessity, and we leave this decision up to you (there are no corresponding tests).

**Note:** Depending on your implementations, you may want to ensure that you call `.compact()` before reshaping arrays. (If this is necessary, you will run into corresponding error messages later in the assignment.)

In [5]:
!python3 -m pytest -l -v -k "nd_backend"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/needle
plugins: anyio-3.5.0
collected 1823 items / 1685 deselected / 138 selected                          

tests/test_nd_backend.py::test_ewise_fn[cpu-shape0-divide] PASSED        [  0%]
tests/test_nd_backend.py::test_ewise_fn[cpu-shape0-subtract] PASSED      [  1%]
tests/test_nd_backend.py::test_ewise_fn[cpu-shape1-divide] PASSED        [  2%]
tests/test_nd_backend.py::test_ewise_fn[cpu-shape1-subtract] PASSED      [  2%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape0-divide] PASSED       [  3%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape0-subtract] PASSED     [  4%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape1-divide] PASSED       [  5%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape1-subtract] PASSED     

In [ ]:
!python3 -m mugrade submit "_0QwrEWYLgzYU25KDG4FM" -k "new_nd_backend"

submit
============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.2.0, pluggy-1.0.0
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 10 items / 9 deselected / 1 selected                                 

tests/test_nd_backend.py 
Submitting new_nd_backend...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
Grader test 9 passed
Grader test 10 passed
Grader test 11 passed
Grader test 12 passed
Grader test 13 passed
Grader test 14 passed
Grader test 15 passed
Grader test 16 passed
Grader test 17 passed
Grader test 18 passed
Grader test 19 passed
Grader test 20 passed
Grader test 21 passed
Grader test 22 passed
Grader test 23 passed
Grader test 24 passed
Grader test 25 passed
Grader test 26 passed
Grader test 27 passed
Grader test 28 passed
Grader test 29 passed
Grader test 30 p

## Part 2: CIFAR-10 dataset [10 points]

Next, you will write support for the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) image classification dataset, which consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. There are 50k training images and 10k test images. 

Start by implementing the `__init__` function in the `CIFAR10Dataset` class. You can read in the link above how to properly read the CIFAR-10 dataset files you downloaded at the beginning of the homework. Also fill in `__getitem__` and `__len__`. Note that the return shape of the data from `__getitem__` should be in order (3, 32, 32).



In [18]:
!python3 -m pytest -l -v -k "cifar10"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1811 deselected / 12 selected                           

tests/test_cifar_ptb_data.py::test_cifar10_dataset[True] PASSED          [  8%]
tests/test_cifar_ptb_data.py::test_cifar10_dataset[False] PASSED         [ 16%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-True-1] PASSED     [ 25%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-True-15] PASSED    [ 33%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-False-1] PASSED    [ 41%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-False-15] PASSED   [ 50%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cuda-True-1] PASSED    [ 58%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cuda-True-15] PASSED  

In [ ]:
!python3 -m mugrade submit "_0QwrEWYLgzYU25KDG4FM" -k "cifar10"

submit
============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.2.0, pluggy-1.0.0
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 10 items / 9 deselected / 1 selected                                 

tests/test_cifar_ptb_data.py 
Submitting cifar10...
filename: data/cifar-10-batches-py/data_batch_1
filename: data/cifar-10-batches-py/data_batch_2
filename: data/cifar-10-batches-py/data_batch_3
filename: data/cifar-10-batches-py/data_batch_4
filename: data/cifar-10-batches-py/data_batch_5
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
Grader test 9 passed
filename: data/cifar-10-batches-py/test_batch
Grader test 10 passed
Grader test 11 passed
Grader test 12 passed
Grader test 13 passed
Grader test 14 passed
Grader test 15 passed
Grader test 16 passed
Grader test 17 pass

## Part 3: Convolutional neural network [40 points]

Here's an outline of what you will do in this task.

In `python/needle/backend_ndarray/ndarray.py`, implement:
- `flip`
- `pad`

In `python/needle/ops.py`, implement (forward and backward):
- `Flip`
- `Dilate`
- `UnDilate`
- `Conv`

In `python/needle/nn.py`, implement:
- `Flatten`
- `Conv`

In `python/apps/models.py`, fill in the `ResNet9` class.  

In `apps/simple_training.py`, fill in:
- `epoch_general_cifar10`,
- `train_cifar10`
- `evaluate_cifar10`

We have provided a `BatchNorm2d` implementation for you as a wrapper around your previous `BatchNorm1d` implementation. 

### Padding ndarrays

Convolution as typically implemented in deep learning libraries cuts down the size of inputs;
e.g., a (1, 32, 32, 3) image convolved with a 3x3 filter would give a (1, 30, 30, c) output.
A way around this is to pad the input ndarray before performing convolution, e.g., pad with zeros to get a (1, 34, 34, 3) ndarray so that the result is (1, 32, 32, 3). 

Padding is also required for the backward pass of convolution.

You should implement `pad` in `ndarray.py` to closely reflect the behavior of `np.pad`.
That is, `pad` should take a tuple of 2-tuples with length equal to the number of dimensions of the array,
where each element in the 2-tuple corresponds to "left padding" and "right padding", respectively.

For example, if `A` is a (10, 32, 32, 8) ndarray (think NHWC), then `A.pad( (0, 0), (2, 2), (2, 2), (0, 0) )` would be a (10, 36, 36, 8) ndarray where the "spatial" dimension has been padded by two zeros on all sides.

In [19]:
!python3 -m pytest -l -v -k "pad_forward"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1821 deselected / 2 selected                            

tests/test_conv.py::test_pad_forward[params0-needle.backend_ndarray.ndarray_backend_cpu] PASSED [ 50%]
tests/test_conv.py::test_pad_forward[params1-needle.backend_ndarray.ndarray_backend_cpu] PASSED [100%]

====================== 2 passed, 1821 deselected in 0.84s ======================


-------------------------------------

### Flipping ndarrays & FlipOp

In [ ]:
import numpy as np
import ctypes

Some utility code for a demonstration below which you can probably ignore. It might be instructive to check out the `offset` function.

In [ ]:
# reads off the underlying data array in order (i.e., offset 0, offset 1, ..., offset n)
# i.e., ignoring strides
def raw_data(X):
    X = np.array(X) # copy, thus compact X
    return np.frombuffer(ctypes.string_at(X.ctypes.data, X.nbytes), dtype=X.dtype, count=X.size)

# Xold and Xnew should reference the same underlying data
def offset(Xold, Xnew):
    assert Xold.itemsize == Xnew.itemsize
    # compare addresses to the beginning of the arrays
    return (Xnew.ctypes.data - Xold.ctypes.data)//Xnew.itemsize

def strides(X):
    return ', '.join([str(x//X.itemsize) for x in X.strides])

def format_array(X, shape):
    assert len(shape) == 3, "I only made this formatting work for ndims = 3"
    def chunks(l, n):
        n = max(1, n)
        return (l[i:i+n] for i in range(0, len(l), n))
    a = [str(x) if x >= 10 else ' ' + str(x) for x in X]
    a = ['(' + ' '.join(y) + ')' for y in [x for x in chunks(a, shape[-1])]]
    a = ['|' + ' '.join(y) + '|' for y in [x for x in chunks(a, shape[-2])]]
    return '  '.join(a)

def inspect_array(X, *, is_a_copy_of):
    # compacts X, then reads it off in order
    print('Data: %s' % format_array(raw_data(X), X.shape))
    # compares address of X to copy_of, thus finding X's offset
    print('Offset: %s' % offset(is_a_copy_of, X))
    print('Strides: %s' % strides(X))

In order to implement the backwards pass of 2D convolution, we will (probably) need a function which _flips_
axes of NDArray. We say "probably" because you could probably cleverly implement your convolution forward
function to avoid this. However, we think it is easiest to think about this if you have the ability to "flip" the kernel along its vertical and horizontal dimensions.

We will try to build up your intuition for the "flip" operation below in order to help you figure out how to implement it in `ndarray.py`. To do that, we explore numpy's `np.flip` function below. One thing to note is that
`flip` is typically implemented by using negative strides and changing the _offset_ of the underlying array.

For example, flipping an array on _all_ of its axes is equivalent to reversing the array. In this case, you can imagine that we would want all the strides to be negative, and the offset to be the length of the array (to start at the end of the array and "stride" backwards).

Since we did not explicitly support negative strides in our implementation for the last homework, we will merely call `NDArray.make` with them to make our "flipped" array and then immediately call `.compact()`. Other than changing unsigned ints to signed ints in a few places, we suspect your existing `compact` function should not have to change at all to accommodate negative strides. In the .cc and .cu files we distributed, we have already changed the function signatures to reflect this.

Alternatively, you could simply implement `flip` in the CPU backend by copying memory, which you _may_ find more intuitive. We suggest following our mini tutorial below to keep your implementation Python-focused, since we believe it involves approximately the same amount of effort to implement it slightly more naively in C.

Use this array as reference for the other examples:

In [ ]:
A = np.arange(1, 25).reshape(3, 2, 4)
inspect_array(A, is_a_copy_of=A)

Data: |( 1  2  3  4) ( 5  6  7  8)|  |( 9 10 11 12) (13 14 15 16)|  |(17 18 19 20) (21 22 23 24)|
Offset: 0
Strides: 8, 4, 1


We have put brackets around each axis of the array. Notice that for this array, the offset is 0 and the strides are all positive.

----------------------------------------------------------

See what happens when you flip the array along the last axis below. 
Note that the `inspect_array` function compacts the array after flipping it, so you can see the
"logical" order of the data, and the offset is calculated by comparing the address of the **non**-compacted
flipped array with that of `is_copy_of`, i.e., the array `A` we looked at above.

That is, we are looking at how numpy calculates the strides and offset for flipped arrays in order
to copy this behavior in our own implementation.

In [ ]:
A

array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8]],

       [[ 9, 10, 11, 12],
        [13, 14, 15, 16]],

       [[17, 18, 19, 20],
        [21, 22, 23, 24]]])

In [ ]:
inspect_array(np.flip(A, (2,)), is_a_copy_of=A)

Data: |( 4  3  2  1) ( 8  7  6  5)|  |(12 11 10  9) (16 15 14 13)|  |(20 19 18 17) (24 23 22 21)|
Offset: 3
Strides: 8, 4, -1


So flipping the last axis reverses the order of the elements within each 4-dimensional "cell", as you can see above. The stride corresponding to the axis we flipped has been negated. And the offset is 3 -- this makes sense, e.g., because we want the new "first" element of the array to be 4, which was at index 3 in `A`.

In [ ]:
inspect_array(np.flip(A, (1,)), is_a_copy_of=A)

Data: |( 5  6  7  8) ( 1  2  3  4)|  |(13 14 15 16) ( 9 10 11 12)|  |(21 22 23 24) (17 18 19 20)|
Offset: 4
Strides: 8, -4, 1


Again for the middle axis: we negate the middle stride, and the offset is 4, which seems reasonable since we now want the first element to be 5, which was at index 4 in the original array `A`. 

In [ ]:
inspect_array(np.flip(A, (0,)), is_a_copy_of=A)

Data: |(17 18 19 20) (21 22 23 24)|  |( 9 10 11 12) (13 14 15 16)|  |( 1  2  3  4) ( 5  6  7  8)|
Offset: 16
Strides: -8, 4, 1


Try to infer the more general algorithm for computing the offset given the axis to flip.

----------------------------------------------------------------------------------------------------------

Observe what happens when we flip _all_ axes.

In [ ]:
inspect_array(np.flip(A, (0,1,2)), is_a_copy_of=A)

Data: |(24 23 22 21) (20 19 18 17)|  |(16 15 14 13) (12 11 10  9)|  |( 8  7  6  5) ( 4  3  2  1)|
Offset: 23
Strides: -8, -4, -1


As mentioned earlier, the offset is then sufficient to point to the last element of the array, and this is just the "reverse order" version of `A`.

When we flip just axes 1 and 0...

In [ ]:
inspect_array(np.flip(A, (0,1)), is_a_copy_of=A)

Data: |(21 22 23 24) (17 18 19 20)|  |(13 14 15 16) ( 9 10 11 12)|  |( 5  6  7  8) ( 1  2  3  4)|
Offset: 20
Strides: -8, -4, 1


The offset is 20. Looking back on our previous offset computations, do you notice something?

-------------------

With this exploration of numpy's ndarray flipping functionality, which uses negative strides and a custom offset,
try to implement `flip` in `ndarray.py`. You also must implement "flip" forward and backward functions in `ops.py`; note that these should be extremely short.

**Important:** You should call NDArray.make with the new strides and offset, and then immediately `.compact()` this array. The resulting array is then copied and has positive strides. We want this (less-than-optimal) behavior because we did not account for negative strides in our previous implementation. _Aside:_ If you want, consider where/if negative strides break your implementation. `__getitem__` definitely doesn't work due to how we processed slices; is there anything else? (_Note_: this isn't graded.)

Also, if you want to instead add a `flip` operator on the CPU/CUDA backends, that's also okay.



In [20]:
!python3 -m pytest -l -v -k "flip"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1783 deselected / 40 selected                           

tests/test_conv.py::test_flip_forward[params0-needle.backend_ndarray.ndarray_backend_cpu] PASSED [  2%]
tests/test_conv.py::test_flip_forward[params0-needle.backend_ndarray.ndarray_backend_cuda] PASSED [  5%]
tests/test_conv.py::test_flip_forward[params1-needle.backend_ndarray.ndarray_backend_cpu] PASSED [  7%]
tests/test_conv.py::test_flip_forward[params1-needle.backend_ndarray.ndarray_backend_cuda] PASSED [ 10%]
tests/test_conv.py::test_flip_forward[params2-needle.backend_ndarray.ndarray_backend_cpu] PASSED [ 12%]
tests/test_conv.py::test_flip_forward[params2-needle.backend_ndarray.ndarray_backend_cuda] PASSED [ 15%]
tests

-------------------------------------

### Dilation


The dilation operator puts zeros between elements of an NDArray. We will need it for computing the backward pass of convolution when the stride of the convolution is greater than 1. As an example, dilation should do the following to a 2x2 matrix when dilated by 1 on both axes:

$$
\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}
\Longrightarrow
\begin{bmatrix}
1 & 0 & 2 & 0 \\
0 & 0 & 0 & 0 \\
3 & 0 & 4 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}
$$

To get some intuition for why we need dilation for the backward pass of strided convolution, consider a  `stride=2`, `padding="same"`, `input_channels=output_channels=8` convolution applied to an input of size (10, 32, 32, 8). The resulting output will be of size (10, 16, 16, 8) due to the stride, and thus `out_grad` will have shape (10, 16, 16, 8). Yet, the gradient of the input needs to, of course, have shape (10, 32, 32, 8) -- so we must need to increase the size of `out_grad` in some way. Consider also that you could implement strided convolution as `Conv(x)[:, ::2, ::2, :]`, i.e., only keeping every other pixel in the spatial dimension.


Implement `Dilate` in `ops.py`. This function takes two additional parameters (in attrs): the `dilation` amount and the `axes` to dilate. You must also implement the corresponding op `UnDilate`, whose forward pass will be used to implement the gradient of `Dilate`. (This is why we do not have to implement `GetItem` and `SetItem` ops, which can be highly inefficient to backprop through without additional optimizations.)

In [21]:
!python3 -m pytest -l -v -k "dilate_forward"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1821 deselected / 2 selected                            

tests/test_conv.py::test_dilate_forward[needle.backend_ndarray.ndarray_backend_cpu] PASSED [ 50%]
tests/test_conv.py::test_dilate_forward[needle.backend_ndarray.ndarray_backend_cuda] PASSED [100%]

====================== 2 passed, 1821 deselected in 0.85s ======================


In [22]:
!python3 -m pytest -l -v -k "dilate"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1797 deselected / 26 selected                           

tests/test_conv.py::test_dilate_forward[needle.backend_ndarray.ndarray_backend_cpu] PASSED [  3%]
tests/test_conv.py::test_dilate_forward[needle.backend_ndarray.ndarray_backend_cuda] PASSED [  7%]
tests/test_conv.py::test_dilate_backward[params0-needle.backend_ndarray.ndarray_backend_cpu] PASSED [ 11%]
tests/test_conv.py::test_dilate_backward[params0-needle.backend_ndarray.ndarray_backend_cuda] PASSED [ 15%]
tests/test_conv.py::test_dilate_backward[params1-needle.backend_ndarray.ndarray_backend_cpu] PASSED [ 19%]
tests/test_conv.py::test_dilate_backward[params1-needle.backend_ndarray.ndarray_backend_cuda] PASSED [ 23%]
tests

---------------------------------------

### Submit new ops (flip/dilation) to mugrade [10 points]

In [ ]:
!python3 -m mugrade submit "_0QwrEWYLgzYU25KDG4FM" -k "new_ops"

submit
============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.2.0, pluggy-1.0.0
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 10 items / 9 deselected / 1 selected                                 

tests/test_conv.py 
Submitting new_ops...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
Grader test 9 passed
Grader test 10 passed
Grader test 11 passed
.

======================= 1 passed, 9 deselected in 19.17s =======================


-----------------

### Convolution forward

Implement the forward pass of 2D multi-channel convolution in `ops.py`. You should probably refer to [this notebook](https://github.com/dlsyscourse/public_notebooks/blob/main/convolution_implementation.ipynb) from lecture, which implements 2D multi-channel convolution using im2col in numpy.

**Note:** Your convolution op should accept tensors in the NHWC format, as in the example above, and weights in the format (kernel_size, kernel_size, input_channels, output_channels).

However, you will need to add two additional features. Your convolution function should accept arguments for `padding` (default 0) and `stride` (default 1). For `padding`, you should simply apply your padding function to the spatial dimensions (i.e., axes 1 and 2). 

Implementing strided convolution should consist of a relatively small set of changes to your plain convolution implementation.

We recommend implementing convolution without stride first, ensuring you pass some of the tests below, and then adding in stride.

In [10]:
import numpy as np
x = np.arange(1, 25).reshape(3,2, 4)
x

array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8]],

       [[ 9, 10, 11, 12],
        [13, 14, 15, 16]],

       [[17, 18, 19, 20],
        [21, 22, 23, 24]]])

In [11]:
import math
from functools import reduce
import operator
def prod(x):
    return reduce(operator.mul, x, 1)

In [23]:
!python3 -m pytest -l -v -k "op_conv and forward"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1789 deselected / 34 selected                           

tests/test_conv.py::test_op_conv[forward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape0-W_shape0-1-0] PASSED [  2%]
tests/test_conv.py::test_op_conv[forward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape1-W_shape1-1-1] PASSED [  5%]
tests/test_conv.py::test_op_conv[forward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape2-W_shape2-1-2] PASSED [  8%]
tests/test_conv.py::test_op_conv[forward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape3-W_shape3-1-0] PASSED [ 11%]
tests/test_conv.py::test_op_conv[forward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape4-W_shape4-1-0] PASSED [ 14%]
tests/test_conv.py::test_op

-----------------

### Convolution backward

Finding the gradients of 2D multi-channel convolution can be technically quite challenging (especially "rigorously"). We will try to provide some useful hints here. Basically, we encourage you to make use of the surprising fact that _whatever makes the dimensions work out is typically right_.

Ultimately, the backward pass of convolution can be done in terms of the convolution operator itself, with some clever manipulations using `flip`, `dilate`, and multiple applications of `transpose` to both the arguments and the results.

In the last section, we essentially implemented convolution as a matrix product: ignoring the various restride and reshape operations, we basically have something like `X @ W`, where `X` is the input and `W` is the weight. We also have `out_grad`, which is the same shape as `X @ W`. Now, you have already implemented the backward pass of matrix multiplication in a previous assignment, and we can use this knowledge to get some insight into the backward pass of convolution. In particular, referencing your matmul backward implementation, you may notice (heuristically speaking here):

`X.grad = out_grad @ W.transpose` \
`W.grad = X.transpose @ out_grad`

Surprisingly enough, things work out if we just assume that these are also convolutions (and now assuming that `out_grad`, `W`, and `X` are tensors amenable to 2D multi-channel convolution instead of matrices):

`X.grad = ≈conv(≈out_grad, ≈W)` \
`W.grad = ≈conv(≈X, ≈out_grad)`

In which the "≈" indicates that you need to apply some additional operators to these terms in order to get the dimensions to work out, such as permuting/transposing axes, dilating, changing the `padding=` argument to the convolution function, or permuting/transposing axes of the resulting convolution.

As we saw on the [last few slides here](https://dlsyscourse.org/slides/conv_nets.pdf) in class, the transpose of a convolution can be found by simply flipping the kernel. Since we're working in 2D instead of 1D, this means flipping the kernel both vertically and horizontally (thus why we implemented `flip`).

Summarizing some hints for both `X.grad` and `W.grad`:

`X.grad`
- The convolution of `out_grad` and `W`, with some operations applied to those
- `W` should be flipped over both the kernel dimensions
- If the convolution is strided, increase the size of `out_grad` with a corresponding dilation
- Do an example to analyze dimensions: note the shape you want for `X.grad`, and think about how you must permute/transpose the arguments and add padding to the convolution to achieve this shape 
    - This padding depends on both the kernel size and the `padding` argument to the convolution

`W.grad`
- The convolution of `X` and `out_grad`, with some operations applied to those
- The gradients of `W` must be accumulated over the batches; how can you make the conv operator itself do this accumulation?
    - Consider turning batches into channels via transpose/permute
- Analyze dimensions: how can you modify `X` and `out_grad` so that the shape of their convolution matches the shape of `W`? You may need to transpose/permute the result.
    - Remember to account for the `padding` argument passed to convolution

General tips
- Deal with strided convolutions last (you should be able to just drop in `dilate` when you've passed most of the tests)
- Start with the case where `padding=0`, then consider changing `padding` arguments
- You can "permute" axes with multiple calls to `transpose`

It might also be useful to skip ahead to nn.Conv, pass the forward tests, and then use both the tests below and the nn.Conv backward tests to debug your implementation.

In [24]:
!python3 -m pytest -l -v -k "op_conv and backward"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1789 deselected / 34 selected                           

tests/test_conv.py::test_op_conv[backward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape0-W_shape0-1-0] PASSED [  2%]
tests/test_conv.py::test_op_conv[backward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape1-W_shape1-1-1] PASSED [  5%]
tests/test_conv.py::test_op_conv[backward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape2-W_shape2-1-2] PASSED [  8%]
tests/test_conv.py::test_op_conv[backward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape3-W_shape3-1-0] PASSED [ 11%]
tests/test_conv.py::test_op_conv[backward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape4-W_shape4-1-0] PASSED [ 14%]
tests/test_conv.py::te

-----------------

### nn.Conv

#### Fixing init._calculate_fans for convolution
Previously, we have implemented Kaiming uniform/normal initializations, where we essentially assigned `fan_in = input_size` and `fan_out = output_size`.
For convolution, this becomes somewhat more detailed, in that you should multiply both of these by the "receptive field size", which is in this case just the product of the kernel sizes -- which in our case are always going to be the same, i.e., $k\times k$ kernels.

**You will need to edit your `kaiming_uniform`, etc. init functions to support multidimensional arrays.** In particular, you should add a new `shape` argument which is then passed to, e.g., the underlying `rand` function.

You can test this below; though it is not _directly_ graded, it must match ours to pass the nn.Conv mugrade tests.

In [25]:
!python3 -m pytest -l -v -k "kaiming_uniform"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1821 deselected / 2 selected                            

tests/test_conv.py::test_init_kaiming_uniform[needle.backend_ndarray.ndarray_backend_cpu] PASSED [ 50%]
tests/test_conv.py::test_init_kaiming_uniform[needle.backend_ndarray.ndarray_backend_cuda] PASSED [100%]

====================== 2 passed, 1821 deselected in 0.94s ======================


#### Implementing nn.Conv

Essentially, nn.Conv is just a wrapper of the convolution operator we previously implemented
which adds a bias term, initializes the weight and bias, and ensures that the padding is set so that the input and output dimensions are the same (in the `stride=1` case, anyways). 

Importantly, nn.Conv should support NCHW format instead of NHWC format. In particular, we think this makes more sense given our current BatchNorm implementation. You can implement this by applying `transpose` twice to both the input and output.  

- Ensure nn.Conv works for (N, C, H, W) tensors even though we implemented the conv op for (N, H, W, C) tensors
- Initialize the (k, k, i, o) weight tensor using Kaiming uniform initialization with default settings
- Initialize the (o,) bias tensor using uniform initialization on the interval $\pm$`1.0/(in_channels * kernel_size**2)**0.5`
- Calculate the appropriate padding to ensure input and output dimensions are the same
- Calculate the convolution, then add the properly-broadcasted bias term if present

You can now test your nn.Conv against PyTorch's nn.Conv2d with the two PyTest calls below.

In [26]:
!python3 -m pytest -l -v -k "nn_conv_forward"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1813 deselected / 10 selected                           

tests/test_conv.py::test_nn_conv_forward[needle.backend_ndarray.ndarray_backend_cpu-4-8-16-3-1] PASSED [ 10%]
tests/test_conv.py::test_nn_conv_forward[needle.backend_ndarray.ndarray_backend_cpu-32-8-16-3-2] PASSED [ 20%]
tests/test_conv.py::test_nn_conv_forward[needle.backend_ndarray.ndarray_backend_cpu-32-8-8-3-2] PASSED [ 30%]
tests/test_conv.py::test_nn_conv_forward[needle.backend_ndarray.ndarray_backend_cpu-32-16-8-3-1] PASSED [ 40%]
tests/test_conv.py::test_nn_conv_forward[needle.backend_ndarray.ndarray_backend_cpu-32-16-8-3-2] PASSED [ 50%]
tests/test_conv.py::test_nn_conv_forward[needle.backend_ndarray.ndarray_backend

In [27]:
!python3 -m pytest -l -v -k "nn_conv_backward"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1809 deselected / 14 selected                           

tests/test_conv.py::test_nn_conv_backward[needle.backend_ndarray.ndarray_backend_cpu-4-1-1-3-1] PASSED [  7%]
tests/test_conv.py::test_nn_conv_backward[needle.backend_ndarray.ndarray_backend_cpu-14-8-16-3-1] PASSED [ 14%]
tests/test_conv.py::test_nn_conv_backward[needle.backend_ndarray.ndarray_backend_cpu-14-8-16-3-2] PASSED [ 21%]
tests/test_conv.py::test_nn_conv_backward[needle.backend_ndarray.ndarray_backend_cpu-14-8-8-3-1] PASSED [ 28%]
tests/test_conv.py::test_nn_conv_backward[needle.backend_ndarray.ndarray_backend_cpu-14-8-8-3-2] PASSED [ 35%]
tests/test_conv.py::test_nn_conv_backward[needle.backend_ndarray.ndarray_bac

-----------------

### Submit nn.Conv to mugrade [20 points]

In [57]:
!python3 -m mugrade submit "_0QwrEWYLgzYU25KDG4FM" -k "conv_forward"

submit
============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1
rootdir: /content/drive/MyDrive/dls_hw4, inifile:
plugins: typeguard-2.7.1
collected 10 items / 9 deselected                                              

tests/test_conv.py 
Submitting conv_forward...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
Grader test 9 passed
Grader test 10 passed
Grader test 11 passed
Grader test 12 passed
Grader test 13 passed
Grader test 14 passed
.

=================== 1 passed, 9 deselected in 19.42 seconds ====================


In [58]:
!python3 -m mugrade submit "_0QwrEWYLgzYU25KDG4FM" -k "conv_backward"

submit
============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1
rootdir: /content/drive/MyDrive/dls_hw4, inifile:
plugins: typeguard-2.7.1
collected 10 items / 9 deselected                                              

tests/test_conv.py 
Submitting conv_backward...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
Grader test 9 passed
Grader test 10 passed
Grader test 11 passed
Grader test 12 passed
Grader test 13 passed
Grader test 14 passed
Grader test 15 passed
Grader test 16 passed
Grader test 17 passed
Grader test 18 passed
.

=================== 1 passed, 9 deselected in 23.75 seconds ====================


------------------------------------------------

### Implementing "ResNet9"

You will now use your convolutional layer to implement a model similar to _ResNet9_, which is known to be a reasonable model for getting good accuracy on CIFAR-10 quickly (see [here](https://github.com/davidcpage/cifar10-fast)). Our main change is that we used striding instead of pooling and divided all of the channels by 4 for the sake of performance (as our framework is not as well-optimized as industry-grade frameworks).

In the figure below, before the linear layer, you should "flatten" the tensor. We have added a module called `Flatten` in `nn.py` that you can complete and use, or you can simply use `.reshape` in the `forward()` method of your ResNet9.

Make sure that you pass the device to all modules in your model; otherwise, you will get errors about mismatched devices when trying to run with CUDA.

<center><img src="https://github.com/dlsyscourse/hw4/blob/main/ResNet9.png?raw=true" alt="ResNet9" style="width: 400px;" /></center>

We have tried to make it easier to pass the tests here than for previous assignments where you have implemented models. In particular, we are just going to make sure it has the right number of parameters and similar accuracy and loss after 1 or 2 batches of CIFAR-10.

In [28]:
!python3 -m pytest -l -v -k "resnet9"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1821 deselected / 2 selected                            

tests/test_conv.py::test_resnet9[needle.backend_ndarray.ndarray_backend_cpu] PASSED [ 50%]
tests/test_conv.py::test_resnet9[needle.backend_ndarray.ndarray_backend_cuda] PASSED [100%]

====================== 2 passed, 1821 deselected in 0.98s ======================


### Submit ResNet9 to mugrade [10 points]

In [65]:
!python3 -m mugrade submit "_0QwrEWYLgzYU25KDG4FM" -k "resnet9"

submit
============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1
rootdir: /content/drive/MyDrive/dls_hw4, inifile:
plugins: typeguard-2.7.1
collected 10 items / 9 deselected                                              

tests/test_conv.py 
Submitting resnet9...
Grader test 1 passed
tcmalloc: large alloc 1228800000 bytes == 0x25aac000 @  0x7f96549e81e7 0x7f965233214e 0x7f965238a745 0x7f965238a878 0x7f965244a604 0x7f965244d8ec 0x7f96525d8bd4 0x4eafcf 0x5d935e 0x7f9652453e6b 0x6076c6 0x55e732 0x55cd91 0x5d8941 0x5da107 0x586de6 0x5d8cdf 0x55ea20 0x5d8868 0x5d8506 0x55f797 0x5d8868 0x5d8506 0x55f797 0x55d078 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997c7 0x4fd2db
Grader test 2 passed
.

=============================== warnings summary ===============================
tests/test_conv.py::submit_resnet9
  /content/drive/MyDrive/dls_hw4/./python/needle/backend_ndarray/ndarray.py:115: VisibleDeprecation

-----------------

Now, you can train your model on CIFAR-10 using the following code. Note that this is likely going to be quite slow, and also  not all that accurate due to the lack of data augmentation. You should expect it to take around 500s per epoch.

In [12]:
import sys
sys.path.append('./python')
sys.path.append('./apps')
import needle as ndl
from models import ResNet9
from simple_training import train_cifar10, evaluate_cifar10

device = ndl.cuda()
train_dataset = ndl.data.CIFAR10Dataset("data/cifar-10-batches-py", train=True)
test_dataset = ndl.data.CIFAR10Dataset("data/cifar-10-batches-py", train=False)
train_dataloader = ndl.data.DataLoader(
         dataset=train_dataset,
         batch_size=256,
         shuffle=True,
         device=device,
         dtype="float32")
test_dataloader = ndl.data.DataLoader(
         dataset=test_dataset,
         batch_size=256,
         shuffle=False,
         device=device,
         dtype="float32")
model = ResNet9(device=device, dtype="float32")
print("---------------CUDA training------------")
train_cifar10(model, train_dataloader, n_epochs=24, optimizer=ndl.optim.SGD,
      lr=0.256, weight_decay=0.00125)
evaluate_cifar10(model, train_dataloader)
print("---------------CUDA testing------------")
evaluate_cifar10(model, test_dataloader)

---------------CUDA training------------
train epoch0: avg_acc: 0.38908, avg_loss: [1.7017603], time cost: 36.562822580337524
train epoch1: avg_acc: 0.49718, avg_loss: [1.3911166], time cost: 37.25927519798279
train epoch2: avg_acc: 0.54312, avg_loss: [1.2674365], time cost: 37.32245421409607
train epoch3: avg_acc: 0.5741, avg_loss: [1.1869293], time cost: 35.87103223800659
train epoch4: avg_acc: 0.60074, avg_loss: [1.1200746], time cost: 37.57653999328613
train epoch5: avg_acc: 0.62038, avg_loss: [1.0667164], time cost: 39.30511546134949
train epoch6: avg_acc: 0.6403, avg_loss: [1.0131098], time cost: 39.83780002593994
train epoch7: avg_acc: 0.65492, avg_loss: [0.9714654], time cost: 34.44371557235718
train epoch8: avg_acc: 0.6675, avg_loss: [0.94136673], time cost: 34.77000713348389
train epoch9: avg_acc: 0.67488, avg_loss: [0.91882634], time cost: 34.89504265785217
train epoch10: avg_acc: 0.68302, avg_loss: [0.8927343], time cost: 35.44233798980713
train epoch11: avg_acc: 0.69022, a

(0.5477, array([1.5374882], dtype=float32))

## Part 4: Recurrent neural network [10 points]

**Note:** In the following sections, you may find yourself wanting to index into tensors, i.e., to use getitem or setitem. However, we have not implemented these for tensors in our library; instead, you should use `stack` and `split` operations.

In `python/needle/nn.py`, implement `RNNCell`.

$h^\prime = \text{tanh}(xW_{ih} + b_{ih} + hW_{hh} + b_{hh})$. If nonlinearity is 'relu', then ReLU is used in place of tanh.

All weights and biases should be initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k=\frac{1}{\text{hidden\_size}}$.

In `python/needle/nn.py`, implement `RNN`.

For each element in the input sequence, each layer computes the following function:

$h_t = \text{tanh}(x_tW_{ih} + b_{ih} + h_{(t-1)}W_{hh} + b_{hh})$

where $h_t$ is the hidden state at time $t$, $x_t$ is the input at time $t$, and $h_{(t-1)}$ is the hidden state of the previous layer at time $t-1$ or the initial hidden state at time $0$. If nonlinearity is 'relu', then ReLU is used in place of tanh.

In a multi-layer RNN, the input $x_t^{(l)}$ of the $l$-th layer ($l \ge 2$) is the hidden state $h_t^{(l-1)}$ of the previous layer. 


In [9]:
!python3 -m pytest -l -v -k "test_rnn_cell"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1695 deselected / 128 selected                          

tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-1-1] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-1-15] PASSED [  1%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-11-1] PASSED [  2%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-11-15] PASSED [  3%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-12-1-1] PASSED [  3%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-12-1-15] PASSED [  4%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-12-11-1] PASSED [  5%]
tests/test_sequence_models.p

In [9]:
!python3 -m pytest -l -v -k "test_rnn"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1183 deselected / 640 selected                          

tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-1-1] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-1-15] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-11-1] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-11-15] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-12-1-1] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-12-1-15] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-12-11-1] PASSED [  1%]
tests/test_sequence_models.p

In [11]:
!python3 -m mugrade submit "_0QwrEWYLgzYU25KDG4FM" -k "rnn"

submit
============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 10 items / 9 deselected / 1 selected                                 

tests/test_sequence_models.py 
Submitting rnn...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
Grader test 9 passed
Grader test 10 passed
Grader test 11 passed
Grader test 12 passed
.

======================= 1 passed, 9 deselected in 16.50s =======================


## Part 5: Long short-term memory network [10 points]
Implement - `Sigmoid`

$\sigma(x) = \frac{1}{1 + \text{exp}(-x)}$

In `python/needle/nn.py`, implement `Sigmoid`, `LSTMCell` and `LSTM`.

\begin{align}
i &= \sigma(xW_{ii} + b_{ii} + hW_{hi} + b_{hi}) \\
f &= \sigma(xW_{if} + b_{if} + hW_{hf} + b_{hf}) \\
g &= \text{tanh}(xW_{ig} + b_{ig} + hW_{hg} + b_{hg}) \\
o &= \sigma(xW_{io} + b_{io} + hW_{ho} + b_{ho}) \\
c^\prime &= f * c + i * g \\
h^\prime &= o * \text{tanh}(c^\prime)
\end{align}

where $\sigma$ is the sigmoid function, and $i$, $f$, $g$, $o$ are the input, forget, cell, and output gates, respectively. 

All weights and biases should be initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k=\frac{1}{\text{hidden\_size}}$.

Now implement `LSTM` in `python/needle/nn.py`, which applies a multi-layer LSTM RNN to an input sequence. For each element in the input sequence, each layer computes the following function:

\begin{align}
i_t &= \sigma(x_tW_{ii} + b_{ii} + h_{(t-1)}W_{hi} + b_{hi}) \\
f_t &= \sigma(x_tW_{if} + b_{if} + h_{(t-1)}W_{hf} + b_{hf}) \\
g_t &= \text{tanh}(x_tW_{ig} + b_{ig} + h_{(t-1)}W_{hg} + b_{hg}) \\
o_t &= \sigma(x_tW_{io} + b_{io} + h_{(t-1)}W_{ho} + b_{ho}) \\
c_t &= f * c_{(t-1)} + i * g \\
h_t &= o * \text{tanh}(c_t)
\end{align},
where $h_t$ is the hidden state at time $t$, $c_t$ is the cell state at time $t$, $x_t$ is the input at time $t$, $h_{(t-1)}$ is the hidden state of the layer at time $t-1$ or the initial hidden state at time $0$, and $i_t$, $f_t$, $g_t$, $o_t$ are the input, forget, cell, and output gates at time $t$ respectively. 

In a multi-layer LSTM, the input $x_t^{(l)}$ of the $l$-th layer ($l \ge 2$) is the hidden state $h_t^{(l-1)}$ of the previous layer.

In [24]:
!python3 -m pytest -l -v -k "test_lstm_cell"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1759 deselected / 64 selected                           

tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-1-1-1] PASSED [  1%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-1-1-15] PASSED [  3%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-1-11-1] PASSED [  4%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-1-11-15] PASSED [  6%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-12-1-1] PASSED [  7%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-12-1-15] PASSED [  9%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-12-11-1] PASSED [ 10%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-T

In [26]:
!python3 -m pytest -l -v -k "test_lstm"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1503 deselected / 320 selected                          

tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-1-1-1] PASSED [  0%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-1-1-15] PASSED [  0%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-1-11-1] PASSED [  0%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-1-11-15] PASSED [  1%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-12-1-1] PASSED [  1%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-12-1-15] PASSED [  1%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-True-12-11-1] PASSED [  2%]
tests/test_sequence_models.py::test_lstm_cell[cpu-True-T

In [27]:
!python3 -m mugrade submit "_0QwrEWYLgzYU25KDG4FM" -k "lstm"

submit
============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 10 items / 9 deselected / 1 selected                                 

tests/test_sequence_models.py 
Submitting lstm...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
Grader test 9 passed
Grader test 10 passed
Grader test 11 passed
Grader test 12 passed
Grader test 13 passed
Grader test 14 passed
Grader test 15 passed
Grader test 16 passed
.

======================= 1 passed, 9 deselected in 30.92s =======================


## Part 6: Penn Treebank dataset [10 points]

In word-level language modeling tasks, the model predicts the probability of the next word in the sequence, based on the words already observed in the sequence. You will write support for the Penn Treebank dataset, which consists of stories from the Wall Street Journal, to train and evaluate a language model on word-level prediction.

In `python/needle/data.py`, start by implementing the `Dictionary` class, which creates a dictionary from a list of words, mapping each word to a unique integer.

Next, we will use this `Dictionary` class to create a corpus from the train and test txt files in the Penn Treebank dataset that you downloaded at the beginning of the notebook. Implement the `tokenize` function in the `Corpus` class to do this.

In order to prepare the data for training and evaluation, you will next implement the `batchify` function. Starting from sequential data, batchify arranges the dataset into columns. For instance, with the alphabet as the sequence and batch size 4, we'd get

```
┌ a g m s ┐
│ b h n t │
│ c i o u │
│ d j p v │
│ e k q w │
└ f l r x ┘
```

These columns are treated as independent by the model, which means that the dependence of e. g. 'g' on 'f' cannot be learned, but allows more efficient batch processing.

Next, implement the `get_batch` function. `get_batch` subdivides the source data into chunks of length `bptt`. If source is equal to the example output of the batchify function, with a bptt-limit of 2, we'd get the following two Variables for i = 0:
```
┌ a g m s ┐ ┌ b h n t ┐
└ b h n t ┘ └ c i o u ┘
```
Note that despite the name of the function, the subdivison of data is not done along the batch dimension (i.e. dimension 1), since that was handled by the batchify function. The chunks are along dimension 0, corresponding to the seq_len dimension in the LSTM or RNN.

In [33]:
import re
lines = ["rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate",
 "a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported"]
l = [re.sub('[^A-Za-z\<\>]+', ' ', line).strip().lower() for line in lines]
l

['rudolph <unk> n years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate',
 'a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than n years ago researchers reported']

In [37]:
!python3 -m pytest -l -v -k "ptb"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1797 deselected / 26 selected                           

tests/test_cifar_ptb_data.py::test_cifar10_dataset[True] PASSED          [  3%]
tests/test_cifar_ptb_data.py::test_cifar10_dataset[False] PASSED         [  7%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-True-1] PASSED     [ 11%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-True-15] PASSED    [ 15%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-False-1] PASSED    [ 19%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-False-15] PASSED   [ 23%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cuda-True-1] PASSED    [ 26%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cuda-True-15] PASSED  

In [38]:
!python3 -m mugrade submit "_0QwrEWYLgzYU25KDG4FM" -k "ptb"

submit
============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 10 items / 8 deselected / 2 selected                                 

tests/test_cifar_ptb_data.py 
Submitting cifar10...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
Grader test 9 passed
Grader test 10 passed
Grader test 11 passed
Grader test 12 passed
Grader test 13 passed
Grader test 14 passed
Grader test 15 passed
Grader test 16 passed
Grader test 17 passed
Grader test 18 passed
.
Submitting ptb...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
Grader test 9 passed
Grader test 10 passed
Grader test 11 passed
Grader t

## Part 7: Training a word-level language model [10 points]

Finally, you will use the `RNN` and `LSTM` components you have written to construct a language model that we will train on the Penn Treebank dataset.

First, in `python/needle/nn.py` implement `Embedding`. Consider we have a dictionary with 1000 words. Then for a word which indexes into this dictionary, we can represent this word as a one-hot vector of size 1000, and then use a linear layer to project this to a vector of some embedding size.

In `apps/models.py`, you can now implement `LanguageModel`. Your language model should consist of 

- An embedding layer (which maps word IDs to embeddings) 
- A sequence model (either RNN or LSTM)
- A linear layer (which outputs probabilities of the next word)

In `apps/simple_training.py` implement `epoch_general_ptb`, `train_ptb`, and `evaluate_ptb`.

In [47]:
import numpy as np
n=20
i=np.arange(16).reshape(2,8)
print(i[1][0])
np.eye(n)[i].shape

8


(2, 8, 20)

In [51]:
a = (1,2,3)
(0, *a)

(0, 1, 2, 3)

In [61]:
!python3 -m pytest -l -v -k "language_model_implementation"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 1823 items / 1311 deselected / 512 selected                          

tests/test_sequence_models.py::test_language_model_implementation[cpu-rnn-1-True-1-1-1-1-1] PASSED [  0%]
tests/test_sequence_models.py::test_language_model_implementation[cpu-rnn-1-True-1-1-1-1-13] PASSED [  0%]
tests/test_sequence_models.py::test_language_model_implementation[cpu-rnn-1-True-1-1-1-2-1] PASSED [  0%]
tests/test_sequence_models.py::test_language_model_implementation[cpu-rnn-1-True-1-1-1-2-13] PASSED [  0%]
tests/test_sequence_models.py::test_language_model_implementation[cpu-rnn-1-True-1-1-15-1-1] PASSED [  0%]
tests/test_sequence_models.py::test_language_model_implementation[cpu-rnn-1-True-1-1-15-1-13] PASSE

In [4]:
!python3 -m pytest -l -v -k "language_model_training"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/needle
plugins: anyio-3.5.0
collected 1823 items / 1821 deselected / 2 selected                            

tests/test_sequence_models.py::test_language_model_training[cpu] FAILED  [ 50%]
tests/test_sequence_models.py::test_language_model_training[cuda] FAILED [100%]

=================================== FAILURES ===================================
______________________ test_language_model_training[cpu] _______________________

device = cpu()

    @pytest.mark.parametrize("device", _DEVICES, ids=["cpu", "cuda"])
    def test_language_model_training(device):
        np.random.seed(0)
        corpus = ndl.data.Corpus("data/ptb", max_lines=20)
        seq_len = 10
        num_examples = 100
        batch_size = 16
        seq_model = 

In [73]:
!python3 -m mugrade submit "_0QwrEWYLgzYU25KDG4FM" -k "language_model"

submit
============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/erostrate9/PycharmProjects/dls_hw4
plugins: anyio-3.5.0
collected 10 items / 9 deselected / 1 selected                                 

tests/test_sequence_models.py 
Submitting language_model...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
Grader test 9 passed
Grader test 10 passed
train epoch0: avg_acc: 0.020833333333333332, avg_loss: 5.8477863073349, time cost: 0.24608230590820312
train epoch1: avg_acc: 0.0625, avg_loss: 5.535320162773132, time cost: 0.1699514389038086
evaluate: avg_acc: 0.0625, avg_loss: 5.357750773429871
Grader test 11 passed
Grader test 12 passed
.

======================= 1 passed, 9 deselected in 26.08s =======================


Now, you can train your language model on the Penn Treebank dataset:

In [2]:
%cd /home/erostrate9/PycharmProjects/dls_hw4
import sys
sys.path.append('./python')
sys.path.append('./apps')
import needle as ndl
from models import LanguageModel
from simple_training import train_ptb, evaluate_ptb

device = ndl.cuda()
corpus = ndl.data.Corpus("data/ptb")
train_data = ndl.data.batchify(corpus.train, batch_size=16, device=device, dtype="float32")
model = LanguageModel(30, len(corpus.dictionary), hidden_size=10, num_layers=2, seq_model='rnn', device=device)
train_ptb(model, train_data, seq_len=1, n_epochs=1, device=device)
evaluate_ptb(model, train_data, seq_len=40, device=device)

/home/erostrate9/PycharmProjects/dls_hw4


KeyboardInterrupt: 

In [ ]:
%cd /home/erostrate9/PycharmProjects/dls_hw4
import sys
sys.path.append('./python')
sys.path.append('./apps')
import needle as ndl
from models import LanguageModel
from simple_training import train_ptb, evaluate_ptb

device = ndl.cuda()
corpus = ndl.data.Corpus("data/ptb")
train_data = ndl.data.batchify(corpus.train, batch_size=16, device=device, dtype="float32")
model = LanguageModel(30, len(corpus.dictionary), hidden_size=10, num_layers=2, seq_model='lstm', device=device)
train_ptb(model, train_data, seq_len=40, n_epochs=5, device=device)
evaluate_ptb(model, train_data, seq_len=40, device=device)

In [8]:
%cd /home/erostrate9/PycharmProjects/needle
import sys
sys.path.append('./python')
sys.path.append('./apps')
import needle as ndl

/home/erostrate9/PycharmProjects/needle


In [21]:
s = (1,2,3,4)
s[:-1]

(1, 2, 3)

In [16]:
import numpy as np
C = ndl.Tensor(np.random.randn(5,10,3), device=ndl.cpu())
D = ndl.Tensor(np.random.randn(5,3,6), device=ndl.cpu())
c = ndl.ops.split(C,0)
d = ndl.ops.split(D,0)

In [18]:
c @ d

TypeError: unsupported operand type(s) for @: 'TensorTuple' and 'TensorTuple'

In [31]:
!python3 -m pytest -l -v -k "batch_matmul"

============================= test session starts ==============================
platform linux -- Python 3.9.15, pytest-7.1.2, pluggy-1.0.0 -- /home/erostrate9/anaconda3/envs/dls/bin/python3
cachedir: .pytest_cache
rootdir: /home/erostrate9/PycharmProjects/needle
plugins: anyio-3.5.0
collected 1845 items / 1823 deselected / 22 selected                           

tests/test_nd_backend.py::test_batch_matmul[cpu-3-3-4-16-16] PASSED      [  4%]
tests/test_nd_backend.py::test_batch_matmul[cpu-5-5-4-8-8] PASSED        [  9%]
tests/test_nd_backend.py::test_batch_matmul[cpu-3-1-2-2-3] PASSED        [ 13%]
tests/test_nd_backend.py::test_batch_matmul[cpu-4-3-3-4-5] PASSED        [ 18%]
tests/test_nd_backend.py::test_batch_matmul[cpu-1-5-5-4-3] PASSED        [ 22%]
tests/test_nd_backend.py::test_batch_matmul[cpu-11-16-15-16-32] PASSED   [ 27%]
tests/test_nd_backend.py::test_batch_matmul[cpu-12-64-64-64-64] PASSED   [ 31%]
tests/test_nd_backend.py::test_batch_matmul[cpu-13-72-2-72-72] PASSED    

In [ ]:
!python3 -m pytest -l -v -k "softmax"